# Title

## Setup

In [1]:
from flask import Flask, request, render_template, url_for
import os
import argparse
import random
import os 
import re
import json 
import linecache
from transformers import pipeline, set_seed
from transformers import BertTokenizer, BertForNextSentencePrediction
import nltk 
from nltk import tokenize
import ssl
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased', return_dict=True)

#Local storage of the conversation data - will be deprecated once the database is in place
storage = []

starters = ["What topics would you like to talk about?", "What are your hobbies?", "Where did you study?"]

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[nltk_data] Downloading package punkt to /Users/amc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Q Generator

In [2]:
generator = pipeline('text-generation', model='gpt2')

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The function below isn't used for now...

## Try it out

In [35]:
def nb_trial_generateNextQ(text):
# text = "What's your name? My name is Alberto."

    storage.append(text)

    if len(starters) > 0: 
        print("SENDING STARTER")
        return {"q":starters.pop()}

    else: 

        text = " ".join(storage[-2:])
        q = generator(text, num_return_sequences=3,max_length=50+len(text))

        #all generated examples 
        allGenerations = ""
        for i in range(3):
            allGenerations = allGenerations +" "+ q[i]['generated_text'][len(text)-4:]

        #Separating all the sentences... 
        sentenceList = nltk.tokenize.sent_tokenize(allGenerations)

        #Filter out questions 
        questionsList = []
        for sentence in sentenceList :
            if "?" in sentence:
                questionsList.append(sentence.strip("\n").strip("\\").strip('"'))

        #Bert evaluation
        bert_filtered_qs = []
        for sentence in questionsList:
            encoding = tokenizer(" ".join(storage[-3:]), sentence, return_tensors='pt')
            outputs = model(**encoding)
            logits = outputs.logits
            bert_filtered_qs.append((logits[0,0].item(), sentence))


        bert_filtered_qs.sort(key=lambda tup: tup[0])


        print(bert_filtered_qs)

        # return {"q":bert_filtered_qs[-1][1]}
        print({"q":bert_filtered_qs[-1][1]})
    

In [ ]:
storage = []

starters = ["What topics would you like to talk about?", "What are your hobbies?", "Where did you study?"]

In [36]:
nb_trial_generateNextQ("What's the best thing you've learned about doing and training as an American? I'm not an American, but I've learned I have the inner willpower to do what it takes to build a better version of myself.")

print("\n#############\n", 
      starters, 
      "\n#############\n", 
      storage)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[(5.492365837097168, "Where's your favorite place on Earth and why do you think it is it would be the most romantic place you could love to be alive?"), (5.694579601287842, 'Any advice that could help you start the process of your goal set?'), (5.900996208190918, "What would that help you do for the rest of the life you've been in the game?"), (6.091255187988281, 'Do you have other training options in mind?'), (6.170355796813965, 'If you do it, what can you bring to the gym that would help you do that?')]
{'q': 'If you do it, what can you bring to the gym that would help you do that?'}

#############
 [] 
#############
 ['What topics would you like to talk about? Tesla cars, philosophy, food and movies.', 'Where did you study? In Italy, in Milan.', "What's your name? My name is Alberto.", 'Nice to meet you. Nice to meey you too.', "But don't you think I'm a spy? I hope not. In any case, I recorded only answers that I'm okay sharing publicly!", "Is there a chance I'd be too serious? I c